In [ ]:
import pandas as pd
import numpy as np
import glasgow_behavior

In [ ]:
#Seperate main GlasgowStudyAll.csv into smaller csvs for analysis
glasgow_behavior.build_csvs("../data/GlasgowStudyAll.csv","../data/subj_status.csv","../csvs")

In [ ]:
def build_nbridge(csv_dir, out_dir, to_drop, name, followup=False):

    #Scan id's
    scans = pd.read_csv(f'{csv_dir}/Scan_Misc.csv')


    #CAARMS
    caarms = pd.read_csv(f'{csv_dir}/CAARMS_baseline.csv')

    #Get only relevant cols
    cols = []
    for col in caarms.columns.to_list():
        if 'final' in col:
            if 'distress' not in col:
                cols.append(col)


    caarms = caarms[['Study_ID'] + cols]
    caarms = caarms.drop(columns=['CAARMS_group_baseline_final_score'])

    caarms = caarms.replace('Not applicable',pd.NA)
    caarms = caarms.replace("-9999",pd.NA)
    caarms = caarms.replace(-9999.0,pd.NA)
    caarms = caarms.replace(" ",pd.NA)

    #Export rows with NA vals
    caarmsNA = caarms[caarms.isna().any(axis=1)]
    caarmsNA.to_csv(f"{csv_dir}/CAARMS_baseline_NA.csv", index=False)

    caarms = caarms.dropna()
    caarms['CAARMS_UTC_severity'] = caarms['CAARMS_UTC_global_baseline_final_score'] * caarms['CAARMS_UTC_frequency_baseline_final_score']
    caarms['CAARMS_NBI_severity'] = caarms['CAARMS_NBI_global_baseline_final_score'] * caarms['CAARMS_NBI_frequency_baseline_final_score']
    caarms['CAARMS_PA_severity'] = caarms['CAARMS_PA_global_baseline_final_score'] * caarms['CAARMS_PA_frequency_baseline_final_score']
    caarms['CAARMS_DS_severity'] = caarms['CAARMS_DS_global_baseline_final_score'] * caarms['CAARMS_DS_frequency_baseline_final_score']
    caarms = caarms.drop(columns=['CAARMS_UTC_global_baseline_final_score','CAARMS_UTC_frequency_baseline_final_score','CAARMS_NBI_global_baseline_final_score','CAARMS_NBI_frequency_baseline_final_score','CAARMS_PA_global_baseline_final_score','CAARMS_PA_frequency_baseline_final_score','CAARMS_DS_global_baseline_final_score','CAARMS_DS_frequency_baseline_final_score'])
    caarms.to_csv(f"{csv_dir}/CAARMS_baseline_final.csv", index=False)

    #BACS
    bacs = pd.read_csv(f'{csv_dir}/BACS_baseline.csv')
    bacs = bacs.replace('Not applicable',pd.NA)
    bacs = bacs.replace("-9999",pd.NA)
    bacs = bacs.replace(-9999.0,pd.NA)
    bacs = bacs.replace(" ",pd.NA)

    #bacs = scans.join(bacs.set_index('Study_ID'), on='Study_ID')

    #Export rows with NA vals
    bacs = bacs[['Study_ID'] + ["BACS_baseline_VerbMemory_total_score","BACS_baseline_DigitSeq","BACS_baseline_Tokens_correctly_in_container","BACS_baseline_SematicFluency","BACS_baseline_LetterFluency_F","BACS_baseline_LetterFluency_S","BACS_baseline_SymbolCoding","BACS_baseline_ToL"]]
    bacsNA = bacs[bacs.isna().any(axis=1)]
    bacsNA.to_csv(f"{csv_dir}/BACS_baseline_NA.csv", index=False)

    #bacs = bacs.dropna()
    bacs = bacs.dropna(thresh=len(bacs.columns)-2, axis=0)
    bacs = bacs.replace(pd.NA,"NA")
    bacs.to_csv(f"{csv_dir}/BACS_baseline_final.csv", index=False)

    #SPIA
    spia = pd.read_csv(f'{csv_dir}/SPIA_Baseline.csv')
    spia = spia.replace('Not applicable',pd.NA)
    spia = spia.replace("-9999",pd.NA)
    spia = spia.replace(-9999.0,pd.NA)
    spia = spia.replace(" ",pd.NA)

    #Get only relevant cols
    vars = ['B1','C5','O3','O7','C2','C3','C4','D3','D4','O1','O2','O8']
    vars13 = ['D5','F2','F3','O4_1','O4_2','O4_3','O4_4','O4_5','O4_6','O4_7','O4_8','O4_9','O4_10']
    vars14 = ['F5','O5_1','O5_2']
    cols1 = []
    cols13 = []
    cols14 = []

    spia_cols = spia.columns.to_list()
    for var in vars:
            for col in spia_cols:
                if 'final' in col:
                    if 'Dist' not in col:
                        if var in col:
                            cols1.append(col)

    for col in spia.columns.to_list():
        for var in vars13:
            if 'final' in col:
                if 'Dist' not in col:
                    if var in col:
                        cols13.append(col)
        for var in vars14:
            if 'final' in col:
                if 'Dist' not in col:
                    if var in col:
                        cols14.append(col)

    spia = spia[['Study_ID'] + cols1 + cols13 + cols14]

    spiaNA = spia[spia.isna().any(axis=1)]

    #7,8,9 not really valid. Replacing with NA
    spia = spia.replace(7.0, pd.NA)
    spia = spia.replace(8.0, pd.NA)
    spia = spia.replace(9.0, pd.NA)

    spia['SPIA_Visual_Perception'] = spia[cols13].sum(axis=1)
    spia['SPIA_Acoustic_Perception'] = spia[cols14].sum(axis=1)
    spia = spia.drop(columns=cols13)
    spia = spia.drop(columns=cols14)
    spiaNA.to_csv(f"{csv_dir}/SPIA_baseline_NA.csv", index=False)

    #spia = spia.dropna(thresh=len(spia.columns)-2, axis=0)
    spia = spia.replace(pd.NA,"NA")
    # GF002 = spia[spia['Study_ID'] == "GF002"]
    # GF002['SPIA_Visual_Perception'] = "NA"
    #spia.loc['GF002','SPIA_Visual_Perception'] = 0
    spia['SPIA_Visual_Perception'] = spia['SPIA_Visual_Perception'].astype(object)
    spia = spia.set_index('Study_ID')
    spia.loc['GF002','SPIA_Visual_Perception'] = "NA"
    spia.to_csv(f"{csv_dir}/SPIA_baseline_final.csv", index=False)

    #IPANAS
    ipanas = pd.read_csv(f'{csv_dir}/IPANAS_baseline.csv')
    ipanas = ipanas[["Study_ID","IPANAS_Interested_baseline","IPANAS_Excited_baseline","IPANAS_Strong_baseline","IPANAS_Enthusiastic_baseline","IPANAS_Proud_baseline","IPANAS_Inspired_baseline","IPANAS_Determined_baseline","IPANAS_Attentive_baseline","IPANAS_Active_baseline","IPANAS_Alert_baseline","IPANAS_Distressed_baseline","IPANAS_Upset_baseline","IPANAS_Guilty_baseline","IPANAS_Scared_baseline","IPANAS_Hostile_baseline","IPANAS_Irritable_baseline","IPANAS_Ashamed_baseline","IPANAS_Nervous_baseline","IPANAS_Jittery_baseline","IPANAS_Afraid_baseline"]]
    ipanas = ipanas.replace('Not applicable',pd.NA)
    ipanas = ipanas.replace("-9999",pd.NA)
    ipanas = ipanas.replace(-9999.0,pd.NA)
    ipanas = ipanas.replace(" ",pd.NA)

    #ipanas = scans.join(ipanas.set_index('Study_ID'), on='Study_ID')

    #Export rows with NA vals
    ipanasNA = ipanas[ipanas.isna().any(axis=1)]
    ipanasNA.to_csv(f"{csv_dir}/IPANAS_baseline_NA.csv", index=False)

    ipanas = ipanas.dropna(thresh=len(ipanas.columns)-2, axis=0)
    ipanas = ipanas.replace(pd.NA,"NA")
    ipanas.to_csv(f"{csv_dir}/IPANAS_baseline_final.csv", index=False)

    #Creating merged df

    groups = pd.read_csv(f'{csv_dir}/Group_Misc.csv')
    merged = scans.join(groups.set_index('Study_ID'), on='Study_ID')

    merged["Group"] = merged["Group"].astype(str)
    merged.loc[merged["Group"] == "0.0", "Group"] = "Control"
    merged.loc[merged["Group"] == "1.0", "Group"] = "Clinical High Risk - Negative"
    merged.loc[merged["Group"] == "2.0", "Group"] = "Clinical High Risk - Positive"
    merged.loc[merged["Group"] == "3.0", "Group"] = "First Episode Psychosis"
    merged.loc[merged["Group"] == "4.0", "Group"] = "Transition to FEP"
    merged.loc[merged["Group"] == "5.0", "Group"] = "Transition to HR"

    merged["Converted"] = merged["Group"] == "Transition to FEP"
    merged.loc[merged["Converted"] == True, "Converted"] = 1
    merged.loc[merged["Converted"] == False, "Converted"] = 0

    merged = merged.join(bacs.set_index('Study_ID'), on='Study_ID')
    merged = merged.join(caarms.set_index('Study_ID'), on='Study_ID')

    if followup:
        #load follow-up data
        f_df = pd.read_csv('../data/caarms_followup_all_delta_more.tsv', sep='\t')
        f_data = f_df[f_df['date'] == '12months']
        f_subj = f_data['participant_id'].to_list()
        merged = merged[merged.Scan_ID.isin(f_subj)]
        f_data_comp = pd.DataFrame()
        f_data_comp['participant_id'] = f_data['participant_id']
        f_data_comp['CAARMS_Composite_Delta'] = f_data[['UTC_Delta', 'NBI_Delta', 'PA_Delta', 'DS_Delta']].sum(axis=1)
        f_data_comp['CAARMS_GAF_Delta'] = f_data['GAF_Delta']
        f_data_comp = f_data_comp.rename(columns={'participant_id':'Scan_ID'})
        #print(f_data)
        merged = merged.merge(f_data_comp, on='Scan_ID', how='left')

    merged = merged.join(spia, on='Study_ID')

    merged = merged[~merged.Scan_ID.isin(to_drop)]


    cbs = merged.dropna()
    cbs = cbs.rename(columns={'Scan_ID':'participant_id'})
    cbs = cbs.drop('Study_ID', axis=1)
    cbs.to_csv(f"{out_dir}/{name}_ALL.tsv", sep='\t', index=False)

    if followup:
        cbs_f = cbs.drop(['CAARMS_Composite_Delta','CAARMS_GAF_Delta'], axis=1)
        cbs_f.to_csv(f"{out_dir}/{name}_ALL_NoFVars.tsv", sep='\t', index=False)
        cbs_of = f_data[['participant_id', 'Group','GAF_Delta', 'UTC_Delta', 'NBI_Delta', 'PA_Delta', 'DS_Delta']]
        cbs_of.to_csv(f"{out_dir}/{name}_ALL_FVars.tsv", sep='\t', index=False)

    cbs_nt = cbs[(cbs['Group'] != 'Transition to FEP') & (cbs['Group'] != 'Transition to HR')]
    cbs_nt.to_csv(f"{out_dir}/{name}_NT.tsv", sep='\t', index=False)

    cbs_t = cbs[(cbs['Group'] == 'Transition to FEP') | (cbs['Group'] == 'Transition to HR')]
    cbs_t.to_csv(f"{out_dir}/{name}_T.tsv", sep='\t', index=False)

qc_fail = ["BKE11","CJY31","CTR13","FCS07","KBL14","MCA13","MMA21","MPA20","RSS08","SPA12","SWE24"]
build_nbridge("../csvs","../data", qc_fail, 'behavior_convert')
qc_fail = ["BKE11","CJY31","CTR13","FCS07","KBL14","MCA13","MMA21","MPA20","RSS08","SPA12","SWE24"]
build_nbridge("../csvs","../data", qc_fail, 'behavior_followup', True)

/var/folders/_f/51yc4jgj2yb300z_hgqpzxq59rm7sh/T/ipykernel_44117/2578905355.py:149: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  merged.loc[merged["Converted"] == True, "Converted"] = 1
/var/folders/_f/51yc4jgj2yb300z_hgqpzxq59rm7sh/T/ipykernel_44117/2578905355.py:149: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  merged.loc[merged["Converted"] == True, "Converted"] = 1


In [ ]:
nb = pd.read_csv('../data/behavior_convert_ALL.tsv', sep='\t')
nb_c = nb[nb['Group'] == 'Control']
nb_nc = nb[nb['Group'] != 'Control']

nb_c.to_csv('../data/behavior_C_convert_ALL.tsv', sep='\t', index=False)
nb_nc.to_csv('../data/behavior_NC_convert_ALL.tsv', sep='\t', index=False)

convert = nb[['Converted']]
convert.to_csv('../data/converter_d.csv', header=False, index=False)

In [ ]:
just_caarms = pd.read_csv('../data/behavior_baseline_CAARMS.tsv', sep='\t')
just_caarms

,participant_id,Group,Converted,CAARMS_GAF_Score_baseline_final_score,CAARMS_UTC_severity,CAARMS_NBI_severity,CAARMS_PA_severity,CAARMS_DS_severity
0,BDA15,Clinical High Risk - Positive,0,69.0,0.0,8.0,4.0,8.0
1,CMH10,Clinical High Risk - Positive,0,58.0,12.0,15.0,15.0,12.0
2,APS28,Transition to FEP,1,52.0,9.0,20.0,8.0,15.0
3,JSA14,Clinical High Risk - Positive,0,63.0,15.0,8.0,3.0,8.0
4,HSS25,Transition to FEP,1,60.0,15.0,16.0,20.0,0.0
...,...,...,...,...,...,...,...,...
187,LTE22,Clinical High Risk - Negative,0,58.0,0.0,0.0,0.0,0.0
188,JBR31,Clinical High Risk - Negative,0,63.0,0.0,6.0,0.0,0.0
189,CBA31,Clinical High Risk - Negative,0,80.0,0.0,0.0,0.0,0.0
190,BWH17,Clinical High Risk - Negative,0,78.0,2.0,6.0,6.0,10.0
